In [ ]:
import argparse
import sys
import os

import random

import time
import datetime

from collections import Counter
import numpy as np

import shutil
import inspect

import gc

import re

import keras
from keras import models
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, BatchNormalization
from keras import optimizers

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

from keras import backend as K

import tensorflow as tf

from IPython.display import display
from PIL import Image

import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix



In [ ]:
# Creates directory, if directory exists removes if remove parameter is set to True 
def create_directory(directory_path, remove=False):
    if remove and os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
            os.mkdir(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
    else:
        try:
            os.mkdir(directory_path)
        except:
            print("Could not create directory: ", directory_path)
            return False
        
    return True

# Removes directory, if directory exists 
def remove_directory(directory_path):
    if os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
        
    return True

In [3]:
# print date and time for given type of representation
def date_time(x):
    if x==1:
        print('Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
    if x==2:    
        print('Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now()))
    if x==3:  
        print('Date now: %s' % datetime.datetime.now())
    if x==4:  
        print('Date today: %s' % datetime.date.today())  

In [4]:
# prints a integer for degugging
def debug(x):
    print("-"*40, x, "-"*40)  

In [5]:
# Removes everything except alphabetical and selected characters from name string
def name_correct(name):
    return re.sub(r'[^a-zA-Z,:]', ' ', name).title()

In [6]:
# reset tensorflow graph tp free up memory and resource allocation 
def reset_graph(model=None):
    try:
        del model
    except:
        return False
    
    tf.reset_default_graph()
    
    K.clear_session()
    
    gc.collect()
    
    return True


# reset callbacks 
def reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint=None
    reduce_lr = None
    early_stopping = None
    tensorboard = None
    

In [7]:
reset_graph(model)
reset_callbacks()

NameError: name 'model' is not defined

In [8]:
# Configure input/ output directory
# Configure training, validation, testing directory

input_directory = r"data/input/"
output_directory = r"data/output/"

training_dir = input_directory+ r"train"
validation_dir = input_directory+ r"val"
testing_dir = input_directory+ r"test"

classes = os.listdir(training_dir)
print(classes)
classes = ['NORMAL', 'PNEUMONIA']
classes

['NORMAL', 'PNEUMONIA']


['NORMAL', 'PNEUMONIA']

In [9]:
target_size = (299,299)
# target_size = (150,150)

rescale=1./255.0
# batch_size = 32

batch_size = 32


class_mode='categorical'

datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=rescale)     

train_generator = datagen.flow_from_directory(
    training_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size)


validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size)    

test_datagen = ImageDataGenerator(rescale=rescale)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1)

Found 5216 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [10]:
# from sklearn.utils import class_weight

# class_weight = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)
# class_weight

In [11]:
#--------------------------------------------------------------------------------------------------------------------#
# Model Directory
#--------------------------------------------------------------------------------------------------------------------#
model_dir=output_directory + r"models/"+time.strftime('%Y-%m-%d %H-%M-%S')+"/"
# model_dir=output_directory + r"models/"+time.strftime('%Y%m%d%H%M%S')+"/"
#--------------------------------------------------------------------------------------------------------------------#
# Log Directory
#--------------------------------------------------------------------------------------------------------------------#
# log_dir=output_directory + r"logs/"+time.strftime('%Y%m%d%H%M%S')
log_dir=output_directory + r"logs/"+time.strftime('%Y-%m-%d %H-%M-%S')
#--------------------------------------------------------------------------------------------------------------------#



# Create Output Directory (Model and Log)
#--------------------------------------------------------------------------------------------------------------------#
create_directory(model_dir, remove=True)
create_directory(log_dir, remove=True)
#--------------------------------------------------------------------------------------------------------------------#


#--------------------------------------------------------------------------------------------------------------------#
# Model File Name Configuration
#------------------------------------------------------------------------------------------------------#

#--------------------------------------------------------------------------------------------------------------------#
# Initial Trainning Model Filename 
#----------------------------------------------------------------------------------------------------#
init_model_file=model_dir+"base-"+"{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
#--------------------------------------------------------------------------------------------------------------------#

#--------------------------------------------------------------------------------------------------------------------#
# Trainning Model Filename 
#----------------------------------------------------------------------------------------------------#
model_file=model_dir+"{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
#--------------------------------------------------------------------------------------------------------------------#

#--------------------------------------------------------------------------------------------------------------------#
# Retrainning Model Filename 
#----------------------------------------------------------------------------------------------------#
retrain_model_file=model_dir+"retrain-{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
#--------------------------------------------------------------------------------------------------------------------#

In [12]:
reset_graph(model)

NameError: name 'model' is not defined

In [13]:
def model():
    base_model = InceptionV3(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    #     x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    predictions = Dense(2, activation='softmax')(x) 

    model = Model(inputs=base_model.input, outputs=predictions)


    for layer in base_model.layers:
        layer.trainable = False

    model.summary()
    
    return model


In [ ]:
checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_loss',
    verbose=0,
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto',
    period=1)

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     min_delta=0, 
#     patience=0, 
#     verbose=0, 
#     mode='auto', 
#     baseline=None, 
#     restore_best_weights=False)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1,
    restore_best_weights=True)




tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=batch_size)


reduce_lr = ReduceLROnPlateau(monitor='val_loss')

In [ ]:
# from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
# from keras.callbacks import Callback
    
    
# from keras import backend as K

# def check_units(y_true, y_pred):
#     if y_pred.shape[1] != 1:
#         y_pred = y_pred[:,1:2]
#         y_true = y_true[:,1:2]
#     return y_true, y_pred

# def precision(y_true, y_pred):
#     y_true, y_pred = check_units(y_true, y_pred)
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def recall(y_true, y_pred):
#     y_true, y_pred = check_units(y_true, y_pred)
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall


# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
#     y_true, y_pred = check_units(y_true, y_pred)
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))



# def macro_f1(y_true, y_pred):
#     y_pred = K.round(y_pred)
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(f1)

In [ ]:
# import numpy as np
# from keras.callbacks import Callback
# from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
# class Metrics(Callback):
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []

#     def on_epoch_end(self, epoch, logs={}):
#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
#         _val_f1 = f1_score(val_targ, val_predict)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
#         self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
#         print (" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
#         return

# metrics = Metrics()

In [ ]:
date_time(1)

# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]


In [ ]:
date_time(1)
model = model()
date_time(1)

In [ ]:
optimizer=optimizers.Adam()
loss='categorical_crossentropy'
# metrics=['accuracy']


# model.compile(optimizer, loss=loss, metrics=metrics)
model.compile(loss=loss,
              optimizer= optimizer,
              metrics=['accuracy'])

date_time(1)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs=30,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    class_weight=class_weight)


date_time(1)


In [ ]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [ ]:
date_time(1)

cur_model_file = r"data\output\models\2018-12-08 17-16-53\03-val_acc-0.94-val_loss-0.16.hdf5"

cur_model = keras.models.load_model(cur_model_file, custom_objects={'accuracy':'accuracy', 'precision':precision, 'recall':recall, 'f1':f1})

# cur_model = keras.models.load_model(cur_model_file)
optimizer=optimizers.Adam(lr=0.00001)
loss='categorical_crossentropy'




metrics=['accuracy', precision, recall, f1_score]
cur_model.compile(optimizer, loss=loss, metrics=metrics)

cur_model.compile(
    loss=loss,
    optimizer= optimizer,
    metrics=metrics)




history = cur_model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs=30,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    class_weight=class_weight)


date_time(1)


In [ ]:
model_dir = "data/output/models/"

dirs = os.listdir(model_dir)
for i in range(len(dirs)):
    print(i, dirs[i])

In [ ]:
current_dir = model_dir+dirs[2]+"/"

print(current_dir)
cur_models = os.listdir(current_dir)
for i in range(len(cur_models)):
    print(i, cur_models[i])

In [ ]:
cur_model = cur_models[5]
model_file = current_dir+cur_model
print(model_file)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    classes = classes,
    class_mode=class_mode,
    batch_size=1)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=target_size,
    classes = classes,
    class_mode=class_mode,
    batch_size=1)

In [ ]:
model = keras.models.load_model(model_file, custom_objects={'accuracy':'accuracy', 'precision':precision, 'recall':recall, 'f1':f1})

In [ ]:
date_time(1)
# result = model.evaluate_generator(generator=test_generator, steps=100, verbose=1)
result = model.evaluate_generator(generator=validation_generator, steps=len(validation_generator), verbose=1)
        

In [ ]:
result

In [ ]:
loss = result[0]
accuracy = result[1]*100
precision = result[2]*100
recall = result[3]*100
f1_score = result[4]*100

print("%s%.2f%s"% ("Accuracy: ", accuracy, "%"))
print("%s%.2f%s"% ("Precision: ", precision, "%"))
print("%s%.2f%s"% ("Recall: ", recall, "%"))
print("%s%.2f%s"% ("F1 - score: ", f1_score, "%"))
print("%s%.2f"% ("Loss: ", loss))

date_time(1)

In [ ]:
date_time(1)

# y_preds = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
y_preds = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
        
y_classes = y_preds.argmax(axis=-1)
date_time(1)

In [ ]:
CM = confusion_matrix(test_generator.classes, y_classes)

fig, ax = plot_confusion_matrix(
    conf_mat=CM ,  
    figsize=(10,8), 
    hide_ticks=True,cmap=plt.cm.Blues)
# plt.title("Model File: "+cur_model)
plt.xticks(range(len(classes)), classes, fontsize=14)
plt.yticks(range(len(classes)), classes, fontsize=14)
# cm_name = "data/output/figures/CM-"+cur_model[:-5]+".png"
# print(cm_name)
# plt.savefig("data/output/figures/CM"+cur_model+".png", dpi=None, facecolor='w', edgecolor='w',
#         orientation='portrait', papertype=None, format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         frameon=None, metadata=None)
plt.show()

In [ ]:
CM = confusion_matrix(test_generator.classes, y_classes)

cls_report_print = classification_report(test_generator.classes, y_classes, target_names=classes)

# cls_report = classification_report(test_generator.classes, y_classes, target_names=classes, output_dict=True)

# if print_report: 
print(cls_report_print)


In [ ]:
import numpy as np
from sklearn import metrics

y_true=test_generator.classes
y_pred = y_preds.argmax(-1) 

x=metrics.accuracy_score(y_true, y_pred)

x=metrics.accuracy_score(y_true, y_pred, normalize=False)

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)

x=metrics.auc(fpr, tpr)


x=metrics.average_precision_score(y_true, y_pred) 


x=metrics.balanced_accuracy_score(y_true, y_pred)

x=metrics.brier_score_loss(y_true, 1-y_pred, pos_label=1) 

x=metrics.f1_score(y_true, y_pred, average='micro')
print(x)
x=metrics.f1_score(y_true, y_pred, average='macro')
x=metrics.f1_score(y_true, y_pred, average='weighted')
x=metrics.f1_score(y_true, y_pred, average=None)



# x=metrics.fbeta_score(y_true, y_pred, average='macro', beta=0.5)

# x=metrics.fbeta_score(y_true, y_pred, average='micro', beta=0.5)

# x=metrics.fbeta_score(y_true, y_pred, average='weighted', beta=0.5)

# x=metrics.fbeta_score(y_true, y_pred, average=None, beta=0.5)

# x=metrics.hamming_loss(y_true, y_pred)

x=metrics.roc_auc_score(y_true, y_pred)
print(x)



